In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union

from transformers.models.roberta.modeling_roberta import (
    RobertaPreTrainedModel,
    RobertaLayer,
    RobertaEmbeddings,
    RobertaPooler,
    RobertaClassificationHead
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    SequenceClassifierOutput
)

In [2]:
FLAG = True
LAMBDA = 0.5

In [3]:
class RobertaMixerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([RobertaLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )

In [4]:
class RobertaMixerModel(RobertaPreTrainedModel):
    """
    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in *Attention is
    all you need*_ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz
    Kaiser and Illia Polosukhin.
    To behave as an decoder the model needs to be initialized with the `is_decoder` argument of the configuration set
    to `True`. To be used in a Seq2Seq model, the model needs to initialized with both `is_decoder` argument and
    `add_cross_attention` set to `True`; an `encoder_hidden_states` is then expected as an input to the forward pass.
    .. _*Attention is all you need*: https://arxiv.org/abs/1706.03762
    """

    _keys_to_ignore_on_load_missing = [r"position_ids"]

    # Copied from transformers.models.bert.modeling_bert.BertModel.__init__ with Bert->Roberta
    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config

        self.embeddings = RobertaEmbeddings(config)
        self.encoder = RobertaMixerEncoder(config)

        self.pooler = RobertaPooler(config) if add_pooling_layer else None

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)


            
    # Copied from transformers.models.bert.modeling_bert.BertModel.forward
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        r"""
        encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


In [5]:
class RobertaMixerForSequenceClassification(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.roberta = RobertaMixerModel(config, add_pooling_layer=False)
        
        if FLAG:
            self.mixup_dense = nn.Linear(config.hidden_size, config.hidden_size)
            self.mixup_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
            self.mixup_dropout = nn.Dropout(config.hidden_dropout_prob)

        self.classifier = RobertaClassificationHead(config)

        # Initialize weights and apply final processing
        self.post_init()

    
    
    def forward(
        self,
        input_ids_1: Optional[torch.LongTensor] = None,
        attention_mask_1: Optional[torch.FloatTensor] = None,
        token_type_ids_1: Optional[torch.LongTensor] = None,
        input_ids_2: Optional[torch.LongTensor] = None,
        attention_mask_2: Optional[torch.FloatTensor] = None,
        token_type_ids_2: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels_1: Optional[torch.LongTensor] = None,
        labels_2: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs_1 = self.roberta(
            input_ids_1,
            attention_mask=attention_mask_1,
            token_type_ids=token_type_ids_1,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output_1 = outputs_1[0]
        
        # Mixup train
        if (input_ids_2 is not None) and (attention_mask_2 is not None) and (labels_2 is not None):
            
            outputs_2 = self.roberta(
                input_ids_2,
                attention_mask=attention_mask_2,
                token_type_ids=token_type_ids_2,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
            sequence_output_2 = outputs_2[0]

            sequence_output = (LAMBDA * sequence_output_1) + ((1 - LAMBDA) * sequence_output_2)

            if FLAG:
                sequence_output = self.mixup_dense(sequence_output)
                sequence_output = self.mixup_layernorm(sequence_output)
                sequence_output = self.mixup_dropout(sequence_output)

            logits = self.classifier(sequence_output)

            loss = None
            loss_fct = nn.CrossEntropyLoss()
            loss = (LAMBDA * loss_fct(logits.view(-1, self.num_labels), labels_1.view(-1))) + ((1 - LAMBDA) * loss_fct(logits.view(-1, self.num_labels), labels_2.view(-1)))

        # Mixup eval
        else:
            logits = self.classifier(sequence_output_1)
            loss = None
            if labels_1 is not None:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels_1.view(-1))
         
        # Return logits, loss, and hidden states
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [6]:
MODEL = RobertaMixerForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaMixerForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaMixerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaMixerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaMixerForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['mixup_layernorm.

In [7]:
import pandas as pd
import numpy as np
import random
from transformers import RobertaTokenizerFast

from tqdm import tqdm
import torch

from torch.utils.data import (
    Dataset,
    ConcatDataset,
    DataLoader
)

In [8]:
TOKENIZER = RobertaTokenizerFast.from_pretrained('roberta-base')

In [9]:
df = pd.read_csv('/projects/ogma3/atharvak/Data_Cartography/datasets/imdb/imdb_categorized.csv')
df

,idx,text,label,category
0,0,I rented I AM CURIOUS-YELLOW from my video sto...,0,ambiguous
1,1,"""I Am Curious: Yellow"" is a risible and preten...",0,easy
2,2,If only to avoid making this type of film in t...,0,easy
3,3,This film was probably inspired by Godard's Ma...,0,easy
4,4,"Oh, brother...after hearing about this ridicul...",0,none
...,...,...,...,...
24995,24995,A hit at the time but now better categorised a...,1,ambiguous
24996,24996,I love this movie like no other. Another time ...,1,none
24997,24997,This film and it's sequel Barry Mckenzie holds...,1,none
24998,24998,'The Adventures Of Barry McKenzie' started lif...,1,ambiguous


In [10]:
df['category'].value_counts()

ambiguous    8250
easy         8250
none         8042
hard          458
Name: category, dtype: int64

In [11]:
df = df[df['category'] != 'ambiguous'].reset_index(drop=True)
df

,idx,text,label,category
0,1,"""I Am Curious: Yellow"" is a risible and preten...",0,easy
1,2,If only to avoid making this type of film in t...,0,easy
2,3,This film was probably inspired by Godard's Ma...,0,easy
3,4,"Oh, brother...after hearing about this ridicul...",0,none
4,5,I would put this at the top of my list of film...,0,easy
...,...,...,...,...
16745,24993,With the mixed reviews this got I wasn't expec...,1,none
16746,24994,"Very smart, sometimes shocking, I just love it...",1,easy
16747,24996,I love this movie like no other. Another time ...,1,none
16748,24997,This film and it's sequel Barry Mckenzie holds...,1,none


In [13]:
def get_mixup_data(data, mixup_type='random_same'):
    if mixup_type == 'random_same':
        data_easy = data[data['category'] == 'easy'].reset_index(drop=True)
        temp_easy = data_easy[['idx', 'label', 'category']].copy().rename(columns={"idx": "idx_2", "label": "label_2", "category": "category_2"}).sample(frac=1).reset_index(drop=True)
        data_easy = pd.concat([data_easy, temp_easy], axis=1)
        
        data_hard = data[data['category'] == 'hard'].reset_index(drop=True)
        temp_hard = data_hard[['idx', 'label', 'category']].copy().rename(columns={"idx": "idx_2", "label": "label_2", "category": "category_2"}).sample(frac=1).reset_index(drop=True)
        data_hard = pd.concat([data_hard, temp_hard], axis=1)
        
        return pd.concat([data_easy, data_hard]).sample(frac=1).reset_index(drop=True)
    
    elif mixup_type == 'random_opposite':
        data_easy = data[data['category'] == 'easy'].reset_index(drop=True)
        data_hard = data[data['category'] == 'hard'].reset_index(drop=True)
        
        easy_tuple = list(zip(data_easy['idx'].tolist(), data_easy['label'].tolist(), data_easy['category'].tolist()))
        hard_tuple = list(zip(data_hard['idx'].tolist(), data_hard['label'].tolist(), data_hard['category'].tolist()))
        
        hard4easy = random.choices(hard_tuple, weights=np.ones(len(hard_tuple)), k=len(data_easy))
        easy4hard = random.choices(easy_tuple, weights=np.ones(len(easy_tuple)), k=len(data_hard))

        hard4easy = pd.DataFrame(hard4easy, columns=['idx_2', 'label_2', 'category_2'])
        data_easy = pd.concat([data_easy, hard4easy], axis=1)
        
        easy4hard = pd.DataFrame(easy4hard, columns=['idx_2', 'label_2', 'category_2'])
        data_hard = pd.concat([data_hard, easy4hard], axis=1)
        
        return pd.concat([data_easy, data_hard]).sample(frac=1).reset_index(drop=True)

In [14]:
random_same_df = get_mixup_data(df, mixup_type='random_same')
random_same_df

,idx,text,label,category,idx_2,label_2,category_2
0,10844,"Let me put it another way: balls. Or, how abou...",0,easy,5950,0,easy
1,17567,Bill Paxton stars in and directs this highly o...,1,easy,9231,0,easy
2,11499,"This film is so bad, you can't imagine. The ac...",0,easy,11093,0,easy
3,9592,This movie seemed like it was going to be bett...,0,easy,10621,0,easy
4,1021,"Eddie Murphy's ""Delirious"" is completely and t...",0,easy,11229,0,easy
...,...,...,...,...,...,...,...
8703,16162,I was impressed that I could take my 5 year ol...,1,easy,5966,0,easy
8704,5876,I'll start by apologizing to filmmakers everyw...,0,easy,5639,0,easy
8705,3744,"Don't get me wrong, I'm a huge fan of many of ...",0,easy,18981,1,easy
8706,8784,How Disney can you get? Preppy rich girls act ...,0,easy,8261,0,easy


In [15]:
# random_opposite_df = get_mixup_data(df, mixup_type='random_opposite')
# random_opposite_df

In [20]:
INPUT_COLUMN = 'text'
DATA_COLUMN = 'category'
OUTPUT_COLUMN = 'label'
MAX_LEN = 128

In [21]:
# class BaseDataLoader(Dataset):
    
#     def __init__(
#         self, 
#         tokenizer,
#         data: pd.DataFrame,
#         data_type: str
#     ):
#         self.data = data
#         self.data_type = data_type
#         self.data = self.data[self.data[DATA_COLUMN] == self.data_type].reset_index(drop=True)
#         self.tokenizer = tokenizer
#         self.tokenized_data = tokenizer.batch_encode_plus(
#             self.data[INPUT_COLUMN].tolist(),
#             max_length=MAX_LEN,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask=True,
#             add_special_tokens=True,
#             return_token_type_ids=True,            
#             return_tensors='pt'
#         )
        
        
        
#     def __len__(
#         self
#     ):
#         return len(self.data)
    
    
    
#     def __getitem__(
#         self,
#         index: int
#     ):
#         data = {
#             'data_type': self.data_type,
#             'input_ids_1': self.tokenized_data['input_ids'][index].flatten(),
#             'attention_mask_1': self.tokenized_data['attention_mask'][index].flatten(),
#             'token_type_ids_1': self.tokenized_data['token_type_ids'][index].flatten(),
#             'labels_1': torch.tensor(self.data.iloc[index][OUTPUT_COLUMN], dtype=torch.long),
#         }
        
#         print(self.data)
#         idx2 = self.data.iloc[index]['idx_2']
#         index2 = self.data[self.data['idx'] == idx2].index[0]        
#         print(self.data.iloc[index]['idx'], index, self.data.iloc[index]['idx_2'], index2, "\n\n")

#         data['input_ids_2'] = self.tokenized_data['input_ids'][index2].flatten()
#         data['attention_mask_2'] = self.tokenized_data['attention_mask'][index2].flatten()
#         data['token_type_ids_2'] = self.tokenized_data['token_type_ids'][index2].flatten()
#         data['labels_2'] = torch.tensor(self.data.iloc[index2][OUTPUT_COLUMN], dtype=torch.long)
#         return data

In [22]:
class BaseDataLoader(Dataset):
    
    def __init__(
        self, 
        tokenizer,
        data: pd.DataFrame,
        data_type: str
    ):
        self.data = data
        self.data_type = data_type
        self.data = self.data[self.data[DATA_COLUMN] == self.data_type].reset_index(drop=True)
        self.tokenizer = tokenizer
        self.tokenized_data = tokenizer.batch_encode_plus(
            self.data[INPUT_COLUMN].tolist(),
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_token_type_ids=True,            
            return_tensors='pt'
        )
        
        
        
    def __len__(
        self
    ):
        return len(self.data)
    
    
    
    def __getitem__(
        self,
        index: int
    ):
        data = {
            'data_type': self.data_type,
            'input_ids_1': self.tokenized_data['input_ids'][index].flatten(),
            'attention_mask_1': self.tokenized_data['attention_mask'][index].flatten(),
            'token_type_ids_1': self.tokenized_data['token_type_ids'][index].flatten(),
            'labels_1': torch.tensor(self.data.iloc[index][OUTPUT_COLUMN], dtype=torch.long),
        }
        
        idx2 = self.data.iloc[index]['idx_2']
        index2 = self.data[self.data['idx'] == idx2].index[0]

#         text = self.data.iloc[index]['text']
#         text2 = self.data[self.data['idx'] == idx2]['text'].tolist()[0]

#         print(self.data.iloc[index]['idx'], text, idx2, "text: ", text2, "\n\n")

        data['input_ids_2'] = self.tokenized_data['input_ids'][index2].flatten()
        data['attention_mask_2'] = self.tokenized_data['attention_mask'][index2].flatten()
        data['token_type_ids_2'] = self.tokenized_data['token_type_ids'][index2].flatten()
        data['labels_2'] = torch.tensor(self.data.iloc[index2][OUTPUT_COLUMN], dtype=torch.long)
        return data

In [23]:
easy_data = BaseDataLoader(
    tokenizer=TOKENIZER,
    data=random_same_df,
    data_type='easy'
)

In [24]:
hard_data = BaseDataLoader(
    tokenizer=TOKENIZER,
    data=random_same_df,
    data_type='hard'
)

In [25]:
final_dataset = ConcatDataset([easy_data, hard_data])

In [26]:
data_loader = DataLoader(
    final_dataset,
    batch_size=32,
    shuffle=False
)

In [27]:
index = 0
idx2 = random_same_df[random_same_df['idx'] == random_same_df.iloc[index]['idx_2']].index[0]        
print(random_same_df.iloc[index]['idx'], index, random_same_df.iloc[index]['idx_2'], idx2)

10844 0 5950 2819


In [28]:
for step, batch in enumerate(tqdm(data_loader, desc="data Iteration")):
    data_type = batch['data_type']
    input_ids_1, attention_mask_1, token_type_ids_1, labels_1 = batch['input_ids_1'], batch['attention_mask_1'], batch['token_type_ids_1'], batch['labels_1']
    input_ids_2, attention_mask_2, token_type_ids_2, labels_2 = batch['input_ids_2'], batch['attention_mask_2'], batch['token_type_ids_2'], batch['labels_2']

    print(input_ids_1.shape)
    print(attention_mask_1.shape)
    print(token_type_ids_1.shape)
    print(labels_1.shape)
    
    print(input_ids_2.shape)
    print(attention_mask_2.shape)
    print(token_type_ids_2.shape)
    print(labels_2.shape)
    
    print(len(data_type))
    break

data Iteration:   0%|                                                                                                       | 0/273 [00:00<?, ?it/s]

10844 Let me put it another way: balls. Or, how about bollo*ks. This is truly awful, more embarrassing than those it attempts to satirise. Julia Roberts is a skilled actress, and usually her work is of the highest standard. This movie is so lacking in direction even she struggles to look proficient. Normally she is the consummate professional, yet I swear that in her eyes, there were signs of bewilderment and despair.<br /><br />The one thing that might have rescued this move was the idea about the director (Chris Walken) turning the movie into a secret documentary about the actors. Unfortunately, that theme wasn't explored to it's full potential. Too little, too late.<br /><br />Zeta Jones was wooden, Cusack was Cusack, and Crystal should stick to acting. The two talented ex-Buffy stars had different experiences - Green hopelessly mis-cast, and Balfour under-used. Well done to Julia for just about preventing this from being the worst movie ever made. 5950 text:  How has this piece of 

In [29]:
for i in range(10):
    idx2 = random_same_df.iloc[i]['idx_2']
    index2 = random_same_df[random_same_df['idx'] == idx2].index[0]
    
    text = random_same_df.iloc[i]['text']
    text2 = random_same_df[random_same_df['idx'] == idx2]['text'].tolist()[0]
    
    print(random_same_df.iloc[i]['idx'], text, idx2, "text: ", text2, "\n\n")

10844 Let me put it another way: balls. Or, how about bollo*ks. This is truly awful, more embarrassing than those it attempts to satirise. Julia Roberts is a skilled actress, and usually her work is of the highest standard. This movie is so lacking in direction even she struggles to look proficient. Normally she is the consummate professional, yet I swear that in her eyes, there were signs of bewilderment and despair.<br /><br />The one thing that might have rescued this move was the idea about the director (Chris Walken) turning the movie into a secret documentary about the actors. Unfortunately, that theme wasn't explored to it's full potential. Too little, too late.<br /><br />Zeta Jones was wooden, Cusack was Cusack, and Crystal should stick to acting. The two talented ex-Buffy stars had different experiences - Green hopelessly mis-cast, and Balfour under-used. Well done to Julia for just about preventing this from being the worst movie ever made. 5950 text:  How has this piece of 

In [16]:
input_ids_1

tensor([[    0,  3463, 32326,  ...,     1,     1,     1],
        [    0,   627,   822,  ...,     1,     1,     1],
        [    0,  3245,   295,  ...,     1,     1,     1],
        ...,
        [    0,  1322,  1690,  ...,     1,     1,     1],
        [    0, 17330,     8,  ...,     1,     1,     1],
        [    0,   102, 17846,  ...,     1,     1,     1]])

In [17]:
input_ids_2

tensor([[    0,   405,  1072,  ...,     1,     1,     1],
        [    0, 37945,  1951,  ...,     1,     1,     1],
        [    0,  7424,    45,  ...,     1,     1,     1],
        ...,
        [    0,  9056, 11926,  ...,     1,     1,     1],
        [    0, 33844, 13510,  ...,     1,     1,     1],
        [    0,   354,   818,  ...,     1,     1,     1]])

idx                                 12062
text           exceedingly memorable one 
label                                   1
data_type                            easy
idx_2                               48976
label_2                                 0
data_type_2                          easy
Name: 0, dtype: object